<!-- Title Slide -->

<table>
<tr>
<td style="width:40%; vertical-align:middle; text-align:center;">
  <img src="https://maartengr.github.io/BERTopic/logo.png" alt="Topic Modeling Illustration" width="300"/>
</td>
<td style="width:60%; vertical-align:middle; padding-left:20px;">

  # 📝 Topic Modeling with BERTopic
  ## Session: Text Embeddings

  <br>
  <br>
  <span style="font-size:1.2em; color:gray;">
    Michael Jantscher · TU Graz · Know Center Research GmbH
  </span>

</td>
</tr>
</table>


# 👋 Michael Jantscher

<img src="https://dhgraz.github.io/clariah2025-dse-ml/images/artists/profil_michael.jpg" alt="Michael Jantscher" width="250"/>

**PhD Student** - TU Graz <br>
**Researcher** - Know Center Research GmbH

---

### 🧠 Focus Areas
- Natural Language Processing (NLP) in medical & clinical domains
- Causal reasoning in healthcare and (neuro)radiology
- Agentic AI systems for decision support and research workflows


# 📌 What is a Text Embedding Vector ?- Formal Definition
* Numerical representation of text (words, sentences or documents) in a multi-dimensional space
* Captures meaning and context
* Semantic similar words/sentences/documents -> vectors closer together

In [1]:
from IPython.display import HTML
HTML("""
<style>
.reveal .slides section { text-align: left !important; }
.reveal h1, .reveal h2, .reveal h3, .reveal p, .reveal table { text-align: left !important; }
.reveal table th, .reveal table td { text-align: left !important; }
</style>
""")

In [40]:
import pandas as pd
from sentence_transformers import SentenceTransformer
st_model_small = SentenceTransformer('all-minilm-l6-v2')

sample_string = "I really like this summer school!"

sample_string_embedding = st_model_small.encode(sample_string)
df = pd.DataFrame({
    "text": [sample_string],
    "embedding": [sample_string_embedding]
})
df

,text,embedding
0,I really like this summer school!,"[-0.057146158, -0.053543467, 0.045457065, 0.01..."


# 💡 Uses Cases of Text Embedding Vectors

| **Use Case**                   | **Example** |
|--------------------------------|-------------|
| **Semantic Search**         | Search “doctor” → find content on “physicians” or “healthcare providers” even without exact keywords |
| **Recommendation Systems**  | Suggest similar research papers, movies, or products based on descriptions or reviews |
| **Sentiment Analysis**      | Understand tone (positive/negative/neutral) beyond simple keywords in tweets or reviews |
| **Clustering & Topic Modeling** | Group thousands of news articles or support tickets by topic automatically |
| **Chatbots & Virtual Assistants** | Improve NLU so bots answer contextually, not just by keyword |
| **Fraud Detection**        | Spot unusual or suspicious text patterns in financial or insurance claims |

---



# 🦖 Pre-Embedding Area
📌 **Definition:**
Early NLP approaches represented text as **sparse, high-dimensional vectors**.
Each dimension corresponded to a **unique word or token**, with no sense of meaning or context.

---

### Bag-of-Words (BoW)
- Represents documents as a **vector of word counts**
- Ignores grammar, order, and semantics
- Example:
  `"I like NLP"` → `[1, 1, 1, 0, 0, ...]`

---

### TF-IDF (Term Frequency – Inverse Document Frequency)
- Adjusts raw counts to emphasize **rare, informative words** and downweight common words
- Example: “the” → low weight, “quantum” → high weight

---

#### TF-IDF Formula

$$TF\text{-}IDF(t, d) = TF(t, d) \times \log\left(\frac{N}{DF(t)}\right)$$

Where:
- (TF(t, d)\): Frequency of term \(t\) in document \(d\)
- \(DF(t)\): Number of documents containing \(t\)
- \(N\): Total number of documents


In [41]:
# Load parlamint dataset
df_parlamint = pd.read_csv("../../materials/parlamint/parlamint-it-is-2022.txt", sep="\t")
df_parlamint_subset = df_parlamint.head(1000).copy(deep=True)
df_parlamint

# Group sentence by utterance (=Parent_ID)
df_parlamint_grouped = (df_parlamint.groupby(["Parent_ID"])["Text"]
                        .apply(lambda s: " ".join(s))
                        .reset_index(name="utterance_text"))

In [47]:
df_parlamint.head(5)

,ID,Parent_ID,Text
0,ParlaMint-IS_2022-01-17-20.seg2.1,ParlaMint-IS_2022-01-17-20.u1,President of the United States reports:
1,ParlaMint-IS_2022-01-17-20.seg3.1,ParlaMint-IS_2022-01-17-20.u1,"I have decided, according to the proposal of t..."
2,ParlaMint-IS_2022-01-17-20.seg4.1,ParlaMint-IS_2022-01-17-20.u1,"Arrange sites, January 11th, 2022."
3,ParlaMint-IS_2022-01-17-20.seg6.1,ParlaMint-IS_2022-01-17-20.u1,Katrín Jakobsdóttir's daughter.
4,ParlaMint-IS_2022-01-17-20.seg7.1,ParlaMint-IS_2022-01-17-20.u1,Presidential Letters for a meeting of the Gene...


In [46]:
df_parlamint_grouped.head(5)

,Parent_ID,utterance_text
0,ParlaMint-IS_2022-01-17-20.u1,President of the United States reports: I have...
1,ParlaMint-IS_2022-01-17-20.u10,"Before the weekend, an article by Stefánssonar..."
2,ParlaMint-IS_2022-01-17-20.u11,"I read this decision in Perconte, which is not..."
3,ParlaMint-IS_2022-01-17-20.u12,"In fact, this is shown in the letter quoted by..."
4,ParlaMint-IS_2022-01-17-20.u13,"Yes, that's right. That's right. A senator who..."


In [48]:
# Take a single utterance from the dataset
sample_utterance = df_parlamint[df_parlamint["Parent_ID"] == "ParlaMint-IS_2022-01-17-20.u1"]["Text"]
print("\n".join([e for e in sample_utterance]))

President of the United States reports:
I have decided, according to the proposal of the prime minister, that the Council should meet for an extended meeting on Monday, January 17, 2022 p.m. 3:00.
Arrange sites, January 11th, 2022.
Katrín Jakobsdóttir's daughter.
Presidential Letters for a meeting of the General Assembly for a subsequent meeting on January 17, 2022
I'd like to use this opportunity here after reading this letter and offer the highest. President and w. Senators welcome to New Year's Parliamentary Conferences.


In [50]:
from src.models.models import CountVectorizerEmbedder

# Adding just the utterance sample as vocabulary
cv_model = CountVectorizerEmbedder(vocabulary=sample_utterance, max_features=5, stop_words='english',
                                   ngram_range=(1, 2))

cv_embeddings = cv_model.embed(sample_utterance)
print(f"Number features: {len(cv_model.embedding_model.get_feature_names_out())}", cv_model.embedding_model.get_feature_names_out())
print(f"Shape embedding array: {cv_embeddings.toarray().shape}")
df_cv_output = pd.DataFrame(columns=cv_model.embedding_model.get_feature_names_out(), data=cv_embeddings.toarray())
df_cv_output

Call 'transform' only...
Number features: 5 ['17 2022' '2022' 'january' 'january 17' 'meeting']
Shape embedding array: (6, 5)


,17 2022,2022,january,january 17,meeting
0,0,0,0,0,0
1,1,1,1,1,1
2,0,1,1,0,0
3,0,0,0,0,0
4,1,1,1,1,2
5,0,0,0,0,0


In [51]:
from src.models.models import TfIdfEmbedder

# Adding just the utterance sample as vocabulary
tfidf_model = TfIdfEmbedder(vocabulary=sample_utterance, max_features=5, stop_words='english')
tfidf_embeddings = tfidf_model.embed(sample_utterance)
print(f"Number features: {len(tfidf_model.embedding_model.get_feature_names_out())}", tfidf_model.embedding_model.get_feature_names_out())
print(f"Shape embedding array: {tfidf_embeddings.toarray().shape}")
df_tfidf_output = pd.DataFrame(columns=tfidf_model.embedding_model.get_feature_names_out(), data=tfidf_embeddings.toarray())
df_tfidf_output

Call 'transform' only...
Number features: 5 ['17' '2022' 'january' 'meeting' 'president']
Shape embedding array: (6, 5)


,17,2022,january,meeting,president
0,0.000000,0.000000,0.000000,0.000000,1.0
1,0.540298,0.456156,0.456156,0.540298,0.0
2,0.000000,0.707107,0.707107,0.000000,0.0
3,0.000000,0.000000,0.000000,0.000000,0.0
4,0.394497,0.333062,0.333062,0.788994,0.0
5,0.000000,0.000000,0.000000,0.000000,1.0


## Dense Text Embeddings

📌 **Definition:**
Dense embeddings represent words, sentences, or documents as **low-dimensional, dense vectors**
where similar meanings are **close together in vector space**.

---

### 🟢 Characteristics
- **Low-dimensional** (e.g., 100–1,536 dimensions, not vocab-sized)
- **Dense representation**: most values ≠ 0
- Captures **semantic meaning** and context
- Learned from data via **neural networks**

---

### 🟠️ Brief History
- **Word2Vec (2013)** – First widely used dense word embeddings (Mikolov et al.)
- **GloVe (2014)** – Global Vectors for word representation
- **FastText (2016)** – Adds subword information for better handling of rare words
- **ELMo (2018)** – Contextual word embeddings
- **BERT (2018)** – Contextual embeddings for entire sentences
- **OpenAI / Modern Embeddings (2020s)** – High-quality sentence/document embeddings (e.g., `text-embedding-3-large`)

---

### 🔴 Why It’s Better
- Reduces dimensionality dramatically
- Learns **semantic relationships**
- Powers modern **search, recommendation, and AI assistants**


## Word2Vec: CBOW & Skip-Gram Overview

### 🟢 What is Word2Vec?
- A **shallow, two-layer neural network** that learns word embeddings from context.
- Maps words to **dense vectors** in a continuous space; similar words are **close together**.
- Introduced by Mikolov et al., **2013**.

---

### 🟠️ Architectures

#### 🔹 Continuous Bag-of-Words (CBOW)
- Predicts the **center word** given its context words.
- Fast to train; works well for **frequent words**.

#### 🔹 Skip-Gram
- Predicts **context words** given a center word.
- Performs better for **rare words**, large datasets.

---

### 🔵 Why It Matters
- Captures **semantic relationships**:
  `king - man + woman ≈ queen`
- Major leap from sparse (BoW/TF-IDF) to **dense, meaningful embeddings**.


## Word2Vec: Training & Visual Intuition

### 🟢 Training Workflow
1. **One-hot encoding** for words.
2. **Hidden layer** = embedding lookup table.
3. **Output layer** predicts context words (softmax with negative sampling).
4. Final **hidden layer weights** = embeddings.

---

### 🟠 Visual Intuition

<img src="https://israelg99.github.io/images/2017-03-23-Word2Vec-Explained/word2vec_diagrams.png" alt="Word2Vec Architecture" style="width:40%;">

- Diagram shows the Skip-Gram model predicting context words.
- Only the **embedding layer weights** are retained.

---

### Reference
- Israel G. (2017). *Word2Vec Explained*.
  [https://israelg99.github.io/2017-03-23-Word2Vec-Explained/](https://israelg99.github.io/2017-03-23-Word2Vec-Explained/)


## BERT: Contextual Embeddings

### 🟢 What is BERT?
- **Bidirectional Encoder Representations from Transformers** (2018, Google AI).
- Uses **Transformer architecture** to create **contextual word embeddings**:
  - Each word’s vector depends on **all surrounding words** (left & right context).
- Trained on **masked language modeling** and **next sentence prediction** tasks.

---

### 🟠 Key Innovations
- **Bidirectional**: Unlike Word2Vec/Glove, captures context from both sides.
- **Transformer encoder layers** with self-attention results in rich, deep embeddings.
- **Contextualization**: Same word gets **different vectors** depending on context
  (“bank” in “river bank” vs. “bank account”).

---

### 🔵 Visual Intuition

<img src="images/bert_embeddings.png" alt="BERT Transformer" style="width:40%;">

---

### Reference
- Devlin et al. (2018). *BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding.*
  [https://arxiv.org/abs/1810.04805](https://arxiv.org/abs/1810.04805)


##  Word2Vec: Learning Word Embeddings with CBOW & Skip-Gram

###  What is Word2Vec?
- A **shallow, two-layer neural network** model that learns word embeddings by reconstructing linguistic context—mapping each word to a vector in a continuous, semantic space. Words with similar contexts end up **close together**. :contentReference[oaicite:1]{index=1}

---

###  Model Architectures

#### **Continuous Bag-of-Words (CBOW)**
- **Predicts** a target word using its surrounding context words.
- The model **averages** context word vectors (from a fixed-size window) to forecast the center word. :contentReference[oaicite:2]{index=2}
- **Faster to train**, especially effective for **frequent words**.

#### **Skip-Gram**
- **Predicts** surrounding context words based on a target (center) word.
- Treats each context-target pair as a separate training example—especially effective for **rare words** and with **large corpora**. :contentReference[oaicite:3]{index=3}

---

###  Training Workflow & Embedding Extraction
1. Input words are one-hot encoded.
2. A hidden layer serves as the embedding lookup (weight matrix).
3. Output layer produces probability distributions—using **softmax** or efficient approximations like **negative sampling** or **hierarchical softmax**. :contentReference[oaicite:4]{index=4}
4. After training, the **hidden layer weights** become the final word embeddings for use in downstream tasks.

---

###  Why It Matters
- Captures rich **semantic and syntactic relationships**:
  \( \text{"king"} - \text{"man"} + \text{"woman"} \approx \text{"queen"} \). :contentReference[oaicite:5]{index=5}
- A major leap from sparse, count-based methods—Word2Vec made dense, meaningful text representations both practical and scalable. Word2Vec was first published in **2013**. :contentReference[oaicite:6]{index=6}

---

###  Summary Table

| Architecture | Strengths | Best For |
|--------------|-----------|----------|
| **CBOW**     | Fast training, efficient, good for frequent words | Large datasets where speed matters |
| **Skip-Gram**| Robust with rare words, better embedding granularity | Large corpus with focus on subtle word nuances |

---

###  Visual Intuition
![Word2Vec Diagram](https://israelg99.github.io/images/2017-03-23-Word2Vec-Explained/word2vec_diagrams.png)

---

### Reference
- Israel G. (2017). *Word2Vec Explained* — [https://israelg99.github.io/2017-03-23-Word2Vec-Explained/](https://israelg99.github.io/2017-03-23-Word2Vec-Explained/)

# Why Neural (Dense) Embeddings over Sparse Vector Representation

### Further topics to cover in the lecture
* Chunking is very important!
* How does training look like for the different models?

### Select different sentence-transformer embedding models
* https://huggingface.co/models?pipeline_tag=sentence-similarity

All generated embeddings are based on [sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) <br>
Maybe give a smaller model also a try: [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

### Training
* How could a training look like?
* Which kind of data is necessary therefore?
* Differences between sparse and dense models in terms of training?

### Interesting Tasks
* Find other promising embedding models (sparse and dense) and try them out
    * What about Doc2Vec, word2vec, bag-of-words etc [here](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html)
* Calculate the embedding similarities like in [here](https://huggingface.co/sentence-transformers)
    * There might be different similarity scores dependeing on the embedder (sparse vs dense)

In [1]:
i = 0
i

0